In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                    
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' %i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)

In [3]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()    
# This function can only be called before any Graphs, Ops, or Tensors have been created. 
# It can be used at the beginning of the program for complex migration projects from TensorFlow 1.x to 2.x.

x = tf.compat.v1.placeholder(tf.float32, [None,3072])    
y = tf.compat.v1.placeholder(tf.int64, [None])    

w = tf.compat.v1.get_variable('w', [x.get_shape()[-1], 1], initializer=tf.random_normal_initializer(0,1))
b = tf.compat.v1.get_variable('b', [1], initializer= tf.constant_initializer(0.0))
y_ = tf.matmul(x, w) + b

p_y_1 = tf.nn.sigmoid(y_)
y_reshaped = tf.reshape(y,(-1,1))    
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))
predict = p_y_1 > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
init = tf.compat.v1.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x:batch_data, y:batch_labels})
        if (i+1) %500 == 0:
            print ("[Train] Step: %d, loss: %4.5f, acc:%4.5f" %(i, loss_val, acc_val))
        if (i+1) %5000 ==0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
                test_acc=np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))
            

[Train] Step: 499, loss: 0.33097, acc:0.60000
[Train] Step: 999, loss: 0.30394, acc:0.70000
[Train] Step: 1499, loss: 0.10084, acc:0.90000
[Train] Step: 1999, loss: 0.19999, acc:0.80000
[Train] Step: 2499, loss: 0.24540, acc:0.75000
[Train] Step: 2999, loss: 0.38930, acc:0.60000
[Train] Step: 3499, loss: 0.15199, acc:0.85000
[Train] Step: 3999, loss: 0.17044, acc:0.80000
[Train] Step: 4499, loss: 0.14214, acc:0.85000
[Train] Step: 4999, loss: 0.39017, acc:0.60000
[Test] Step: 5000, acc: 0.81000
[Train] Step: 5499, loss: 0.21992, acc:0.75000
[Train] Step: 5999, loss: 0.37961, acc:0.60000
[Train] Step: 6499, loss: 0.25178, acc:0.75000
[Train] Step: 6999, loss: 0.13443, acc:0.85000
[Train] Step: 7499, loss: 0.10003, acc:0.90000
[Train] Step: 7999, loss: 0.16963, acc:0.80000
[Train] Step: 8499, loss: 0.44999, acc:0.55000
[Train] Step: 8999, loss: 0.10013, acc:0.90000
[Train] Step: 9499, loss: 0.17337, acc:0.80000
[Train] Step: 9999, loss: 0.34952, acc:0.65000
[Test] Step: 10000, acc: 0.812

[Train] Step: 80999, loss: 0.21220, acc:0.80000
[Train] Step: 81499, loss: 0.33383, acc:0.65000
[Train] Step: 81999, loss: 0.15794, acc:0.85000
[Train] Step: 82499, loss: 0.10000, acc:0.90000
[Train] Step: 82999, loss: 0.16757, acc:0.80000
[Train] Step: 83499, loss: 0.00000, acc:1.00000
[Train] Step: 83999, loss: 0.19999, acc:0.80000
[Train] Step: 84499, loss: 0.17881, acc:0.75000
[Train] Step: 84999, loss: 0.10696, acc:0.90000
[Test] Step: 85000, acc: 0.82450
[Train] Step: 85499, loss: 0.20004, acc:0.80000
[Train] Step: 85999, loss: 0.00000, acc:1.00000
[Train] Step: 86499, loss: 0.13843, acc:0.85000
[Train] Step: 86999, loss: 0.25119, acc:0.75000
[Train] Step: 87499, loss: 0.25254, acc:0.75000
[Train] Step: 87999, loss: 0.30445, acc:0.70000
[Train] Step: 88499, loss: 0.20038, acc:0.80000
[Train] Step: 88999, loss: 0.10001, acc:0.90000
[Train] Step: 89499, loss: 0.15043, acc:0.85000
[Train] Step: 89999, loss: 0.10000, acc:0.90000
[Test] Step: 90000, acc: 0.82500
[Train] Step: 90499, l